## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [2]:
%cd /Users/rnikitin/dev/freqrade-experiments/freqtrade

from pathlib import Path
from freqtrade.configuration import Configuration
import pandas as pd

# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files([])
# Optionally (recommended), use existing configuration file
# config = Configuration.from_files(["config.json"])

# Define some constants
config["timeframe"] = "1m"
# Name of the strategy class
config["strategy"] = "EqueumBacktestStrategy"
# Location of the data
data_location = config['datadir']
# Pair to analyze - Only use one pair here
pair = "ETH/USDT"

/Users/rnikitin/dev/freqrade-experiments/freqtrade


In [3]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file 
# backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2020-07-01_20-04-22.json"

In [4]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = 'EqueumBacktestStrategy'
# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


[{'key': 'ETH/USDT', 'trades': 105, 'profit_mean': 0.019822935393013275, 'profit_mean_pct': 1.9822935393013277, 'profit_sum': 2.081408216266394, 'profit_sum_pct': 208.14, 'profit_total_abs': 4879.0878125, 'profit_total': 4.8790878125, 'profit_total_pct': 487.91, 'duration_avg': '2 days, 11:34:00', 'wins': 52, 'draws': 0, 'losses': 53}, {'key': 'TOTAL', 'trades': 105, 'profit_mean': 0.019822935393013275, 'profit_mean_pct': 1.9822935393013277, 'profit_sum': 2.081408216266394, 'profit_sum_pct': 208.14, 'profit_total_abs': 4879.0878125, 'profit_total': 4.8790878125, 'profit_total_pct': 487.91, 'duration_avg': '2 days, 11:34:00', 'wins': 52, 'draws': 0, 'losses': 53}]
['ETH/USDT']
-0.514201631462535
0.35147744662057556
2022-07-19 07:55:00
2022-08-13 03:38:00
[{'key': 'EqueumBacktestStrategy', 'trades': 105, 'profit_mean': 0.019822935393013275, 'profit_mean_pct': 1.9822935393013277, 'profit_sum': 2.081408216266394, 'profit_sum_pct': 208.14, 'profit_total_abs': 4879.0878125, 'profit_total': 4

In [5]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()

pair      exit_reason
ETH/USDT  exit_signal    104
          force_exit       1
Name: exit_reason, dtype: int64

## Plotting daily profit / equity line

In [6]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
import plotly.express as px
import pandas as pd

# strategy = 'SampleStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats['strategy'][strategy]

dates = []
profits = []
for date_profit in strategy_stats['daily_profit']:
    dates.append(date_profit[0])
    profits.append(date_profit[1])

equity = 0
equity_daily = []
for daily_profit in profits:
    equity_daily.append(equity)
    equity += float(daily_profit)


df = pd.DataFrame({'dates': dates,'equity_daily': equity_daily})

fig = px.line(df, x="dates", y="equity_daily")
fig.show()


### Quant Stats

In [8]:
%matplotlib inline
import quantstats as qs

pct_changes = pd.Series(data=equity_daily).pct_change()

print(pct_changes.dtypes)

strategy_returns = pd.Series(data=pct_changes.values.tolist(), index=dates)

qs.reports.full(strategy_returns)

float64


TypeError: unsupported operand type(s) for -: 'str' and 'str'